The contractor passed the rug along to a neighbor. 
Here's the important info for finding them:

> I gave it to this guy who lived in my neighborhood... he was a Cancer born in the year of the Rabbit...


So I'll need to find the customer who:
- lives in Joshua Peterson's neighborhood
- is a Cancer: born between the dates of June 21st and July 22nd ([source](https://www.horoscope.com/zodiac-signs/cancer))
- was born in the year of the Rabbit: any of the following years: 2023, 2011, 1999, 1987, 1975, 1963, and 1951 ([source](https://chinesenewyear.net/zodiac/rabbit/)) 

In [ ]:
uri = "sqlite://data/noahs.sqlite"
query = "SELECT * FROM orders"
#pl.read_database_uri(query=query,uri=uri,engine="adbc")

In [1]:
import polars as pl
import datetime as dt

customers = pl.read_csv("data/noahs-customers.csv",try_parse_dates=True)

In [40]:
# Phone number is from part 2
joshua_peterson_citystatezip = (
    customers.filter(pl.col("phone") == "332-274-4185").select("citystatezip").item()
)
neighbors = customers.filter(pl.col("citystatezip") == joshua_peterson_citystatezip)

june_21_through_31 = (
    pl.col("birthdate").dt.month() == 6,
    pl.col("birthdate").dt.day().is_in(list(range(21, 32))),
)
july_1_through_22 = (
    pl.col("birthdate").dt.month() == 7,
    pl.col("birthdate").dt.day().is_in(list(range(1, 23))),
)

# I'd love to learn out how to OR these two filters together
neighbors_june_cancer = neighbors.filter(june_21_through_31)
neighbors_july_cancer = neighbors.filter(july_1_through_23)

neighbors_cancer = pl.concat(
    [neighbors_june_cancer, neighbors_july_cancer], how="vertical"
)
print(neighbors_cancer.select("name", "phone"))


shape: (2, 2)
┌───────────────┬──────────────┐
│ name          ┆ phone        │
│ ---           ┆ ---          │
│ str           ┆ str          │
╞═══════════════╪══════════════╡
│ Robert Morton ┆ 917-288-9635 │
│ Max Moore     ┆ 315-291-6374 │
└───────────────┴──────────────┘


Wow, that narrows it down to two neighbors who are Cancers!
At this point I could manually inspect their birthdate, but I can just as easily query it.

In [43]:
print(
    neighbors_cancer.filter(
        pl.col("birthdate").dt.year().is_in([2023, 2011, 1999, 1987, 1975, 1963, 1951])
    ).select("name", "phone")
)


shape: (1, 2)
┌───────────────┬──────────────┐
│ name          ┆ phone        │
│ ---           ┆ ---          │
│ str           ┆ str          │
╞═══════════════╪══════════════╡
│ Robert Morton ┆ 917-288-9635 │
└───────────────┴──────────────┘
